In [1]:
# Standard library
import json
import os
import warnings
from pathlib import Path

# Third-party libraries
import numpy as np
import pandas as pd
from matplotlib import colormaps
from matplotlib.colors import Normalize
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

# Show all rows and prevent column truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # Auto-detect width

In [2]:
metric_groups = {
    "Feature_Relevance": [
        "anova_f_mean", 
        "mutual_info_mean"
    ],
    "Local_Overlap": [
        "pca_centroid_distance_pca_centroid_score", 
        "mahalanobis_class_distance_mean"
    ],
    "Boundary_Hardness": [
        "svm_margin_mean", 
        "class_proba_entropy_mean"
    ],
    "Global_Structure": [
        "intrinsic_dimensionality_intrinsic_dimensionality_percent", 
        "calinski_harabasz_calinski_harabasz_score"
    ],
    "Class_Distribution_Separation": [
        "class_confusion_entropy_confusion_entropy", 
        "class_imbalance_normalized_entropy"
    ]
}

In [3]:
# Invert selected metrics (where higher = easier)
metrics_to_invert = {
    "anova_f_mean",
    "mutual_info_mean",
    "svm_margin_mean",
    "pca_centroid_score",
    "mahalanobis_class_distance_mean",
    "calinski_harabasz_calinski_harabasz_score",
    "class_imbalance_normalized_entropy"
}

In [4]:
row_order = [
    "CIC_IDS_2017_Multiclass",
    "CIC_IOT_Dataset2023_Multiclass",
    "IoT_23_Multiclass",
    "IoT_Network_Intrusion_Macro_Multiclass",
    "IoT_Network_Intrusion_Micro_Multiclass",
    "KDD_Cup_1999_Multiclass",
    "UNSW_NB15_Multiclass",
    "BCCC_CIC-BCCC-NRC-ACI-IOT-2023_Multiclass",
    "BCCC_CIC-BCCC-NRC-Edge-IIoTSet-2022_Multiclass",
    "BCCC_CIC-BCCC-NRC-IoMT-2024_Multiclass",
    "BCCC_CIC-BCCC-NRC-IoT-2022_Multiclass",
    "BCCC_CIC-BCCC-NRC-IoT-2023-Original_Training_and_Testing_Multiclass",
    "BCCC_CIC-BCCC-NRC-IoT-HCRL-2019_Multiclass",
    "BCCC_CIC-BCCC-NRC-MQTTIoT-IDS-2020_Multiclass",
    "BCCC_CIC-BCCC-NRC-TONIoT-2021_Multiclass",
    "BCCC_CIC-BCCC-NRC-UQ-IOT-2022_Multiclass",
    "BoT_IoT_Macro_Multiclass",
    "BoT_IoT_Micro_Multiclass",
    "CICAPT_IIoT_Phase1_Macro_Multiclass", # nok (single-class)
    "CICAPT_IIoT_Phase1_Micro_Multiclass", # nok (single-class)
    "CICAPT_IIoT_Phase2_Macro_Multiclass",
    "CICAPT_IIoT_Phase2_Micro_Multiclass",
    "CICEVSE2024_EVSE-A_Macro_Multiclass",
    "CICEVSE2024_EVSE-A_Micro_Multiclass",
    "CICEVSE2024_EVSE-B_Macro_Multiclass",
    "CICEVSE2024_EVSE-B_Micro_Multiclass",
    "CICIoMT2024_Bluetooth_Multiclass",
    "CICIoMT2024_WiFi_and_MQTT_Multiclass",
    "CICIoV2024_Decimal_Macro_Multiclass",
    "CICIoV2024_Decimal_Micro_Multiclass",
    "EDGE-IIOTSET_DNN-EdgeIIoT_Multiclass",
    "EDGE-IIOTSET_ML-EdgeIIoT_Multiclass",
    "MQTT_IoT_IDS2020_BiflowFeatures_Multiclass",
    "MQTT_IoT_IDS2020_PacketFeatures_Multiclass",
    "MQTT_IoT_IDS2020_UniflowFeatures_Multiclass",
    "NIDS_CIC-BoT-IoT_Multiclass",
    "NIDS_CIC-ToN-IoT_Multiclass",
    "NIDS_NF-BoT-IoT_Multiclass",
    "NIDS_NF-BoT-IoT-v2_Multiclass",
    "NIDS_NF-BoT-IoT-v3_Multiclass",
    "NIDS_NF-CICIDS2018-v3_Multiclass",
    "NIDS_NF-CSE-CIC-IDS2018_Multiclass",
    "NIDS_NF-CSE-CIC-IDS2018-v2_Multiclass",
    "NIDS_NF-ToN-IoT_Multiclass",
    "NIDS_NF-ToN-IoT-v2_Multiclass",
    "NIDS_NF-ToN-IoT-v3_Multiclass",
    "NIDS_NF-UNSW-NB15_Multiclass",
    "NIDS_NF-UNSW-NB15-v2_Multiclass",
    "NIDS_NF-UNSW-NB15-v3_Multiclass",
    "NIDS_NF-UQ-NIDS_Multiclass",
    "NIDS_NF-UQ-NIDS-v2_Multiclass",
    "N_BaIoT_Danmini_Doorbell_Multiclass",
    "N_BaIoT_Ecobee_Thermostat_Multiclass",
    "N_BaIoT_Ennio_Doorbell_Multiclass",
    "N_BaIoT_Philips_B120N10_Baby_Monitor_Multiclass",
    "N_BaIoT_Provision_PT_737E_Security_Camera_Multiclass",
    "N_BaIoT_Provision_PT_838_Security_Camera_Multiclass",
    "N_BaIoT_Samsung_SNH_1011_N_Webcam_Multiclass",
    "N_BaIoT_SimpleHome_XCS7_1002_WHT_Security_Camera_Multiclass",
    "N_BaIoT_SimpleHome_XCS7_1003_WHT_Security_Camera_Multiclass",
    "ToN_IoT_IoT_Fridge_Multiclass",
    "ToN_IoT_IoT_GPS_Tracker_Multiclass",
    "ToN_IoT_IoT_Garage_Door_Multiclass",
    "ToN_IoT_IoT_Modbus_Multiclass",
    "ToN_IoT_IoT_Motion_Light_Multiclass",
    "ToN_IoT_IoT_Thermostat_Multiclass",
    "ToN_IoT_IoT_Weather_Multiclass",
    "ToN_IoT_Linux_Disk_Multiclass",
    "ToN_IoT_Linux_Memory_Multiclass",
    "ToN_IoT_Linux_Process_Multiclass",
    "ToN_IoT_Network_Multiclass",
    "ToN_IoT_Windows_10_Multiclass",
    "ToN_IoT_Windows_7_Multiclass"
]

In [5]:
def blend_with_white(rgb, alpha):
    return [1 - alpha * (1 - c) for c in rgb]

def format_and_color_columns(df, color_map_dict={}, alpha=0.0):
    df_colored = df.copy()

    for col in df.columns:
        col_data = df[col]

        # === Step 1: Apply your custom formatting ===
        if pd.api.types.is_float_dtype(col_data):
            if 'time' in col:
                formatted = col_data.map(lambda x: f"{x:,.1f}")
            elif 'size' in col:
                formatted = col_data.map(lambda x: f"{x:,.2f}")
            else:
                formatted = col_data.map(lambda x: f"{x:,.3f}")
        elif pd.api.types.is_integer_dtype(col_data):
            formatted = col_data.map(lambda x: f"{x:,}")
        else:
            formatted = col_data.astype(str)

        # === Step 2: Apply LaTeX color using colormap if specified ===
        if col in color_map_dict and pd.api.types.is_numeric_dtype(col_data):
            cmap = colormaps[color_map_dict[col]]
            valid_mask = col_data.notna()
            norm = Normalize(vmin=col_data[valid_mask].min(), vmax=col_data[valid_mask].max())
    
            # Start with string-typed formatted column
            colored_column = formatted.astype(str).copy()
    
            # Compute blended RGB
            rgba_colors = cmap(norm(col_data[valid_mask]))[:, :3]
            blended_colors = [blend_with_white(rgb, alpha=alpha) for rgb in rgba_colors]
    
            for i, (r, g, b) in zip(col_data[valid_mask].index, blended_colors):
                df_colored.loc[i, col] = (
                    f"\\cellcolor[rgb]{{{r:.3f}, {g:.3f}, {b:.3f}}} {formatted[i]}"
                )
        else:
            df_colored[col] = formatted

    return df_colored

In [6]:
def flatten_metrics_dict(metrics_dict: dict, dataset_id: str, keys_to_include=None) -> dict:
    flat = {"dataset_id": dataset_id}
    for top_key, subdict in metrics_dict.items():
        if isinstance(subdict, dict):
            for sub_key, value in subdict.items():
                flat_key = f"{top_key}_{sub_key}"
                if keys_to_include is None or flat_key in keys_to_include:
                    flat[flat_key] = value
        else:
            if keys_to_include is None or top_key in keys_to_include:
                flat[top_key] = subdict
    return flat

def compute_composite_difficulty_from_dict(metrics_dict: dict, dataset_id: str, min_metrics_per_group=1) -> pd.DataFrame | None:
    all_metrics = [m for group in metric_groups.values() for m in group]
    
    flat_dict = flatten_metrics_dict(metrics_dict, dataset_id, keys_to_include=all_metrics)
    df = pd.DataFrame([flat_dict])
    
    available_metrics = []
    missing_metrics = []

    for metric in all_metrics:
        if metric in df.columns and not pd.isna(df.loc[0, metric]):
            available_metrics.append(metric)
        else:
            missing_metrics.append(metric)

    if missing_metrics:
        print(f"[INFO] {dataset_id}: Missing metrics: {missing_metrics}")
    
    valid_groups = {}
    for group_name, metric_list in metric_groups.items():
        valid_metrics_in_group = [m for m in metric_list if m in available_metrics]
        if len(valid_metrics_in_group) >= min_metrics_per_group:
            valid_groups[group_name] = valid_metrics_in_group
        else:
            print(f"[WARN] {dataset_id}: Group '{group_name}' has only {len(valid_metrics_in_group)} valid metrics")
    
    if len(valid_groups) < 3:
        print(f"[SKIP] {dataset_id}: Only {len(valid_groups)} valid groups, need at least 3")
        return None

    for metric in metrics_to_invert:
        if metric in available_metrics:
            df[metric] = -df[metric]

    # print(f"[INFO] {dataset_id}: Skipping normalization for single dataset")

    for group_name, metric_list in valid_groups.items():
        df[f"{group_name}_difficulty"] = df[metric_list].mean(axis=1)

    group_cols = [f"{g}_difficulty" for g in valid_groups.keys()]
    df["overall_difficulty"] = df[group_cols].mean(axis=1)
    df["metrics_used"] = len(available_metrics)
    df["groups_used"] = len(valid_groups)

    return df

def compute_all_composite_difficulties(root_dir: str, suffix: str = "_complexity_cuml.json", min_metrics_per_group=1) -> pd.DataFrame:
    """
    Loads all complexity metric JSONs from a folder and computes composite difficulty scores.
    More flexible version that handles missing metrics gracefully.
    """
    
    exclude_substrings = {
        "CICAPT_IIoT_Phase1_Macro_Multiclass",
        "CICAPT_IIoT_Phase1_Micro_Multiclass",
        # "ToN_IoT_IoT_Motion_Light_Multiclass"
    }
    
    all_json_paths = [
        path for path in Path(root_dir).rglob(f"*{suffix}")
        if not any(substr in path.stem for substr in exclude_substrings)
    ]

    rows = []

    print(f"Found {len(all_json_paths)} JSON files to process")

    for path in tqdm(all_json_paths, desc="Computing composite difficulties"):
        try:
            with open(path, 'r') as f:
                metrics_dict = json.load(f)

            # Remove metadata keys
            metrics_dict.pop('label_mappings', None)
            metrics_dict.pop('errors', None)

            filename = path.stem
            dataset_id = filename.replace("Output_Multiclass__100_pct__", "").replace("_complexity_cuml", "")

            composite_df = compute_composite_difficulty_from_dict(
                metrics_dict, dataset_id, min_metrics_per_group
            )
            # display(composite_df)
            
            if composite_df is not None:
                rows.append(composite_df)

        except Exception as e:
            print(f"[ERROR] Failed to process {path}: {e}")

    if not rows:
        print("No datasets could be processed")
        return pd.DataFrame()

    result_df = pd.concat(rows, ignore_index=True)
    # display(result_df)

    # Normalize across datasets
    print(f"\nNormalizing metrics across {len(result_df)} datasets...")

    # Determine metrics to normalize
    metric_cols = []
    for group_name, metric_list in metric_groups.items():
        metric_cols.extend(metric_list)

    available_metric_cols = [col for col in metric_cols if col in result_df.columns]

    # Fill NaNs before normalization if needed
    if result_df[available_metric_cols].isnull().values.any():
        print("[WARN] NaNs detected before normalization – filling with 0")
        result_df[available_metric_cols] = result_df[available_metric_cols].fillna(0)

    # Normalize
    if available_metric_cols:
        scaler = MinMaxScaler()
        result_df[available_metric_cols] = scaler.fit_transform(result_df[available_metric_cols])

    # Recompute group difficulties
    for group_name, metric_list in metric_groups.items():
        available_group_metrics = [m for m in metric_list if m in result_df.columns]
        if available_group_metrics:
            result_df[f"{group_name}_difficulty"] = result_df[available_group_metrics].mean(axis=1)

    # Recompute overall difficulty
    group_cols = [f"{group_name}_difficulty" for group_name in metric_groups.keys()
                  if f"{group_name}_difficulty" in result_df.columns]
    if group_cols:
        result_df["overall_difficulty"] = result_df[group_cols].mean(axis=1)

    print(f"Successfully processed {len(result_df)} datasets")
    print(f"Average metrics used per dataset: {result_df['metrics_used'].mean():.1f}")
    print(f"Average groups used per dataset: {result_df['groups_used'].mean():.1f}")

    return result_df

In [7]:
ROOT_DIR = "../../2025-07-05/cuml_v2-20250705T065915Z-1-001"

df_composite = compute_all_composite_difficulties(ROOT_DIR, min_metrics_per_group=1)

display(df_composite)

Found 73 JSON files to process


Computing composite difficulties: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 238.88it/s]

[INFO] ToN_IoT_IoT_Motion_Light_Multiclass: Missing metrics: ['intrinsic_dimensionality_intrinsic_dimensionality_percent', 'calinski_harabasz_calinski_harabasz_score']
[WARN] ToN_IoT_IoT_Motion_Light_Multiclass: Group 'Global_Structure' has only 0 valid metrics

Normalizing metrics across 73 datasets...
[WARN] NaNs detected before normalization – filling with 0
Successfully processed 73 datasets
Average metrics used per dataset: 10.0
Average groups used per dataset: 5.0


,dataset_id,anova_f_mean,mutual_info_mean,pca_centroid_distance_pca_centroid_score,mahalanobis_class_distance_mean,svm_margin_mean,class_proba_entropy_mean,intrinsic_dimensionality_intrinsic_dimensionality_percent,calinski_harabasz_calinski_harabasz_score,class_confusion_entropy_confusion_entropy,class_imbalance_normalized_entropy,Feature_Relevance_difficulty,Local_Overlap_difficulty,Boundary_Hardness_difficulty,Global_Structure_difficulty,Class_Distribution_Separation_difficulty,overall_difficulty,metrics_used,groups_used
0,BCCC_CIC-BCCC-NRC-IoT-HCRL-2019_Multiclass,0.999925,0.743596,1.929483e-01,0.565707,0.782820,0.139039,0.260274,0.984747,8.150677e-01,0.432349,0.871760,0.379328,0.460930,0.622510,6.237083e-01,0.591647,10,5
1,N_BaIoT_Ecobee_Thermostat_Multiclass,0.986693,0.112068,2.999188e-01,0.375063,0.743313,0.050307,0.095652,0.819578,2.742764e-01,0.208510,0.549380,0.337491,0.396810,0.457615,2.413933e-01,0.396538,10,5
2,NIDS_CIC-BoT-IoT_Multiclass,0.999401,0.928500,3.691828e-01,0.259836,0.961606,0.402545,0.139241,0.816073,7.950975e-01,0.311668,0.963950,0.314509,0.682075,0.477657,5.533826e-01,0.598315,10,5
3,CICIoMT2024_WiFi_and_MQTT_Multiclass,0.990342,0.280217,1.468645e-01,0.474082,0.872058,0.205602,0.295455,0.000000,4.873885e-01,0.213296,0.635280,0.310473,0.538830,0.147727,3.503420e-01,0.396530,10,5
4,N_BaIoT_SimpleHome_XCS7_1002_WHT_Security_Came...,0.997953,0.015408,2.077155e-01,0.357016,0.751340,0.085651,0.113043,0.900813,3.609745e-01,0.192608,0.506680,0.282366,0.418496,0.506928,2.767913e-01,0.398252,10,5
5,UNSW_NB15_Multiclass,0.999396,0.936991,6.551139e-02,0.305389,0.616350,0.030661,0.200000,0.950832,1.000000e+00,0.889582,0.968193,0.185450,0.323505,0.575416,9.447912e-01,0.599471,10,5
6,ToN_IoT_IoT_Fridge_Multiclass,1.000000,1.000000,1.513208e-11,0.824563,0.805200,0.935759,1.000000,1.000000,5.560120e-13,0.000000,1.000000,0.412282,0.870480,1.000000,2.780060e-13,0.656552,10,5
7,BoT_IoT_Macro_Multiclass,0.999060,0.908076,1.865326e-01,0.576269,0.928730,0.282183,0.400000,0.826635,5.760841e-01,0.474378,0.953568,0.381401,0.605456,0.613318,5.252310e-01,0.615795,10,5
8,BCCC_CIC-BCCC-NRC-UQ-IOT-2022_Multiclass,0.997722,0.900322,9.828803e-01,0.438227,0.665533,0.013105,0.188406,0.759662,2.838177e-01,0.467635,0.949022,0.710554,0.339319,0.474034,3.757263e-01,0.569731,10,5
9,CICIoV2024_Binary_Macro_Multiclass,0.999999,0.991835,4.213136e-02,0.000000,0.665714,0.010560,0.440000,0.999847,2.081362e-01,0.936382,0.995917,0.021066,0.338137,0.719924,5.722590e-01,0.529460,10,5


In [8]:
# Set index to dataset_id
df_composite_indexed = df_composite.set_index('dataset_id')

# Filter row_order to only include datasets that exist in the DataFrame
valid_order = [d for d in row_order if d in df_composite_indexed.index]

# Reorder using the filtered list
df_ordered = df_composite_indexed.loc[valid_order]

# Export to LaTeX with thousands separator
df_ordered.style.format(thousands=",").to_latex("tables/complexity_metrics.tex")

df_ordered.to_excel('tables/complexity_metrics.xlsx')
df_ordered.to_json('tables/complexity_metrics.json', orient='index')

In [9]:
df_ordered

,anova_f_mean,mutual_info_mean,pca_centroid_distance_pca_centroid_score,mahalanobis_class_distance_mean,svm_margin_mean,class_proba_entropy_mean,intrinsic_dimensionality_intrinsic_dimensionality_percent,calinski_harabasz_calinski_harabasz_score,class_confusion_entropy_confusion_entropy,class_imbalance_normalized_entropy,Feature_Relevance_difficulty,Local_Overlap_difficulty,Boundary_Hardness_difficulty,Global_Structure_difficulty,Class_Distribution_Separation_difficulty,overall_difficulty,metrics_used,groups_used
dataset_id,,,,,,,,,,,,,,,,,,
CIC_IDS_2017_Multiclass,0.999762,0.820646,2.242812e-01,0.178211,0.706356,0.011386,0.171429,0.947096,3.273388e-01,0.816046,0.910204,0.201246,0.358871,0.559262,5.716924e-01,0.520255,10,5
CIC_IOT_Dataset2023_Multiclass,0.995851,0.267017,1.279018e-01,0.391022,0.924679,0.420760,0.333333,0.267942,7.607051e-01,0.143655,0.631434,0.259462,0.672720,0.300638,4.521801e-01,0.463287,10,5
IoT_23_Multiclass,0.999792,0.814665,2.374696e-01,0.733892,0.768897,0.102997,0.500000,0.994567,5.581450e-01,0.764568,0.907228,0.485681,0.435947,0.747283,6.613566e-01,0.647499,10,5
IoT_Network_Intrusion_Macro_Multiclass,0.999991,0.957805,4.064557e-02,0.576206,0.798489,0.177656,0.208333,0.998048,3.325125e-01,0.705845,0.978898,0.308426,0.488073,0.603191,5.191785e-01,0.579553,10,5
IoT_Network_Intrusion_Micro_Multiclass,0.999995,0.952512,4.898892e-02,0.578248,0.803452,0.180365,0.208333,0.999078,1.982160e-01,0.795218,0.976253,0.313619,0.491909,0.603706,4.967170e-01,0.576441,10,5
KDD_Cup_1999_Multiclass,0.990049,0.735637,1.072754e-01,0.548558,0.693825,0.008441,0.200000,0.848160,3.947905e-01,0.790225,0.862843,0.327917,0.351133,0.524080,5.925079e-01,0.531696,10,5
UNSW_NB15_Multiclass,0.999396,0.936991,6.551139e-02,0.305389,0.616350,0.030661,0.200000,0.950832,1.000000e+00,0.889582,0.968193,0.185450,0.323505,0.575416,9.447912e-01,0.599471,10,5
BCCC_CIC-BCCC-NRC-ACI-IOT-2023_Multiclass,0.999927,0.813675,5.126640e-02,0.357712,0.922194,0.506687,0.232877,0.990906,6.163875e-01,0.239575,0.906801,0.204489,0.714440,0.611892,4.279812e-01,0.573121,10,5
BCCC_CIC-BCCC-NRC-Edge-IIoTSet-2022_Multiclass,0.995763,0.787151,2.272014e-01,0.503501,0.680949,0.012546,0.159420,0.743152,3.279459e-01,0.752633,0.891457,0.365351,0.346748,0.451286,5.402897e-01,0.519026,10,5


In [11]:
cols_to_drop = [
    "Feature_Relevance_difficulty", "Local_Overlap_difficulty", 
    "Boundary_Hardness_difficulty", "Global_Structure_difficulty", 
    "Class_Distribution_Separation_difficulty", 
    "metrics_used", "groups_used"
]

df_ordered_pretty = format_and_color_columns(
    df_ordered.drop(columns=cols_to_drop).loc[valid_order],
    color_map_dict={'overall_difficulty': 'RdYlGn_r'},
    alpha=0.5
)

with open("tables/table_2.tex", "w") as f:
    f.write(df_ordered_pretty.to_string())

df_ordered_pretty

/tmp/ipykernel_17604/3793402051.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '\cellcolor[rgb]{0.963, 0.985, 0.825} 0.520' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_colored.loc[i, col] = (


,anova_f_mean,mutual_info_mean,pca_centroid_distance_pca_centroid_score,mahalanobis_class_distance_mean,svm_margin_mean,class_proba_entropy_mean,intrinsic_dimensionality_intrinsic_dimensionality_percent,calinski_harabasz_calinski_harabasz_score,class_confusion_entropy_confusion_entropy,class_imbalance_normalized_entropy,overall_difficulty
dataset_id,,,,,,,,,,,
CIC_IDS_2017_Multiclass,1.000,0.821,0.224,0.178,0.706,0.011,0.171,0.947,0.327,0.816,"\cellcolor[rgb]{0.963, 0.985, 0.825} 0.520"
CIC_IOT_Dataset2023_Multiclass,0.996,0.267,0.128,0.391,0.925,0.421,0.333,0.268,0.761,0.144,"\cellcolor[rgb]{0.813, 0.920, 0.706} 0.463"
IoT_23_Multiclass,1.000,0.815,0.237,0.734,0.769,0.103,0.500,0.995,0.558,0.765,"\cellcolor[rgb]{0.970, 0.695, 0.623} 0.647"
IoT_Network_Intrusion_Macro_Multiclass,1.000,0.958,0.041,0.576,0.798,0.178,0.208,0.998,0.333,0.706,"\cellcolor[rgb]{0.998, 0.916, 0.753} 0.580"
IoT_Network_Intrusion_Micro_Multiclass,1.000,0.953,0.049,0.578,0.803,0.180,0.208,0.999,0.198,0.795,"\cellcolor[rgb]{0.998, 0.928, 0.763} 0.576"
KDD_Cup_1999_Multiclass,0.990,0.736,0.107,0.549,0.694,0.008,0.200,0.848,0.395,0.790,"\cellcolor[rgb]{0.990, 0.996, 0.861} 0.532"
UNSW_NB15_Multiclass,0.999,0.937,0.066,0.305,0.616,0.031,0.200,0.951,1.000,0.890,"\cellcolor[rgb]{0.997, 0.862, 0.708} 0.599"
BCCC_CIC-BCCC-NRC-ACI-IOT-2023_Multiclass,1.000,0.814,0.051,0.358,0.922,0.507,0.233,0.991,0.616,0.240,"\cellcolor[rgb]{0.998, 0.935, 0.769} 0.573"
BCCC_CIC-BCCC-NRC-Edge-IIoTSet-2022_Multiclass,0.996,0.787,0.227,0.504,0.681,0.013,0.159,0.743,0.328,0.753,"\cellcolor[rgb]{0.961, 0.983, 0.821} 0.519"


In [ ]:
# Compute min and max of overall_difficulty
vmin = df_ordered['overall_difficulty'].min()
vmax = df_ordered['overall_difficulty'].max()

# Select and style (don't include 'dataset_id' as it's now the index)
styled_df = df_ordered[['overall_difficulty', 'metrics_used', 'groups_used']].style \
    .background_gradient(subset=['overall_difficulty'], cmap='RdYlGn_r', vmin=vmin, vmax=vmax) \
    .format({'overall_difficulty': '{:.3f}'})

styled_df